## 1: KB

setup

In [3]:
import heapq
import itertools
import random
from collections import defaultdict, Counter
from itertools import chain

In [5]:
def first(iterable, default= None):
    """ return 1st elment in iterable data struct, if empty  None. """
    return next(iter(iterable), default) 

class Expr:
    """ a mathetical expression with an operator and 0 or more args """
    def __init__(self, op, *args):
        self.op = op
        self.args = args # tuple of args

    def __neg__(self):
        return Expr('-', self)

    def __pos__(self):
        return Expr('+', self)

    def __invert__(self):
        return Expr('~', self)

    def __add__(self, rhs):
        return Expr('+', self, rhs)

    def __sub__(self, rhs):
        return Expr('-', self, rhs)

    def __mul__(self, rhs):
        return Expr('*', self, rhs)

    def __pow__(self, rhs):
        return Expr('**', self, rhs)

    def __mod__(self, rhs):
        return Expr('%', self, rhs)

    def __and__(self, rhs):
        return Expr('&', self, rhs)

    def __xor__(self, rhs):
        return Expr('^', self, rhs)

    def __rshift__(self,rhs):
        return Expr('>>', self, rhs)

    def __lshift__(self, rhs):
        return Expr('<<', self, rhs)

    def __floordiv__(self, rhs):
        return Expr('//', self, rhs)

    def __matmul__(self, rhs):
        return Expr('@', self, rhs)

    def __truediv__(self, rhs):
        return Expr('/', self, rhs)

    def __or__(self, rhs):
        """ Allow both P | Q, and P |'==>'| Q. """
        if isinstance(rhs, Expression):
            return Expr('|', self, rhs)
        else:
            return PartialExpr(rhs, self)

    # Reverse operator overloads
    def __radd__(self, lhs):
        return Expr('+', lhs, self)  # noExpr + Expr 

    def __rsub__(self, lhs):
        return Expr('-', lhs, self)

    def __rmul__(self, lhs):
        return Expr('*', lhs, self)

    def __rdiv__(self, lhs):
        return Expr('/', lhs, self)

    def __rpow__(self, lhs):
        return Expr('**', lhs, self)

    def __rmod__(self, lhs):
        return Expr('%', lhs, self)

    def __rand__(self, lhs):
        return Expr('&', lhs, self)

    def __rxor__(self, lhs):
        return Expr('^', lhs, self)

    def __ror__(self, lhs):
        return Expr('|', lhs, self)

    def __rrshift__(self, lhs):
        return Expr('>>', lhs, self)

    def __rlshift__(self, lhs):
        return Expr('<<', lhs, self)

    def __rtruediv__(self, lhs):
        return Expr('/', lhs, self)

    def __rfloordiv__(self, lhs):
        return Expr('//', lhs, self)

    def __rmatmul__(self, lhs):
        return Expr('@', lhs, self)

    def __call__(self, *args): # triggered when you use parentheses on an instance of Expr
        """ if f is a symbol, then f(0) == Expr('f',0). """
        if self.args:
            raise ValueError('can only do a call for a symbol not Expr')
        else:
            return Expr(self.op, *args)

    def __eq__(self, other):
        return isinstance(other, Expr) and self.op == other.op and self.args == other.args

    def __lt__(self, other):
        return isinstance(other, Expr) and str(self) < str(other) # compare alphabetically

    def __hash__(self):
        return hash(self.op) ^ hash(self.args) # to enable Expr to be used as key in dict/set. ^ is xor

    def __repr__(self):
        """ It defines how expressions are printed in a readable format. """
        op = self.op
        args = [str(arg) for arg in self.args]
        if op.isidentifier(): # f(x) or f(x, y) , isidentifier return True if op can be used as variable/func name
            return '{}({})'.format(op, ', '.join(args)) if args else op
        elif len(args) == 1: # *x or -(x+1)
            return op + args[0]
        else: # ( x-y )
            opp = (' '+ op +' ')
            return '(' + opp.join(args) + ')'

# An 'Expression' is either an Expr or a Number.
# Symbol is not an explicit type; it is any Expr with 0 args.

Number = (int, float, complex)
Expression = (Expr, Number)

class PartialExpr:
    """Given 'P |'==>'| Q, first form PartialExpr('==>', P), then combine with Q."""

    def __init__(self, op, lhs):
        self.op, self.lhs = op, lhs

    def __or__(self, rhs):
        return Expr(self.op, self.lhs, rhs)

    def __repr__(self):
        return "PartialExpr('{}', {})".format(self.op, self.lhs)

def Symbol(name):
    return Expr(name)

class defaultkeydict(defaultdict):
    def __missing__(self, key):
        self[key] = result = self.default_facotry(key) # default_factory in our example is Symbol
        return result # return Expr(Symbol)


def expr(x):
    """ Shortcut to create an Expression. x is a str in which:
    - identifiers are automatically defined as Symbols.
    - ==> is treated as an infix |'==>'|, as are <== and <=>.
    If x is already an Expression, it is returned unchanged. Example:
    >>> expr('P & Q ==> Q')
    ((P & Q) ==> Q)
    """
    return eval(expr_handle_infix_ops(x), defaultkeydict(Symbol)) if isinstance(x, str) else x


infix_ops = '==> <== <=>'.split()  # create a list

def expr_handle_infix_ops(x):
    """Given a str, return a new str with ==> replaced by |'==>'|, etc.
    >>> expr_handle_infix_ops('P ==> Q')
    "P |'==>'| Q"
    """
    for op in infix_ops:
        x = x.replace(op, '|' + repr(op) + '|') # repr('==>') returns "'==>'" (a string with quotes).
    return x

def subexpressions(x):
    yield x
    if isinstance(x, Expr):
        for arg in x.args:
            yield from subexpressions(arg)

kb class

In [8]:
class KB:
    def __init__(self, sentence = None):
        if sentence: 
            self.tell(sentence)
            
    def tell(self, sentence):
        raise NotImplementedError
        
    def ask(self, query):
        """ return substituation that makes the query true/{} , or, failing that, return false """
        return first(self.ask_generator(query), default = False)
        
    def ask_generator(self, query):
        raise NotImplementedError
        
    def retract(self, sentence):
        """ remove sentence from the KB """
        raise NotImplementedError

## kb for propsitional logic

In [11]:
class PropKB(KB):
    """ kb for propositional logic """
    def __init__(self, sentence = None):
        super().__init__(sentence)
        self.clauses = []     # is a dijunction of literals

    def tell(self, sentence):
        self.clauses.extend(conjuncts(to_cnf(sentence)))

    def ask_generator(self, query):
        if tt_entails(Expr('&', *self.clauses),query):
            yield {}

    def retract(self, sentence):
        for c in conjuncts(to_cnf(sentence)):
            if c in self.clasuses:
                self.clauses.remove(c)
            
    def ask_if_true(self, query):
        for _ in self.ask_generator(query): # even if it is empty
            return True
        return False


KB agent program [Figure 7.1]

In [14]:
def KBAgentProgram(kb):
    
    steps = itertools.count() # creates an iterator object
    
    def program(percept):
        t = next(steps)
        kb.tell(make_percept_sentence(percept, t))
        action = kb.ask(make_action_query(t))
        kb.tell(make_action_sentence(action, t))
        return action

    def make_percept_sentence(percept, t):
        return Expr('Percept')(percept, t)

    def make_action_query(t):
        return expr('ShouldDo(action, {})'.format(t))

    def make_action_sentence(action, t):
        return Expr('Did')(action[expr('action')], t)

    return program
        

helpers

In [39]:
def is_symbol(s):
    return ininstance(s, str) and s[:1].isalpha()

def is_var_symbol(s):
    """ a logic variable symbol is an intital-lowercase string"""
    return is_symbol(s) and s[0].islower()

def is_prop_symbol(s):
    """ a propositional logic symbols an initial-uppercase string """
    return is_symbol(s) and s[0].isupper()
    
def is_variable(x):
    """ a variable is an Expr with no args and a lowercase symbol as the op. """
    return isinstance(x, Expr) and not x.args and x.op[0].islower()
    
def variables(s):
    """ return a set of variables in an expressin s """
    return {x for x in subexpressions(s) if is_variable(x)} # The for loop starts by calling next() on the generator

def is_deinite_clause(s):
    if is_symbol(s.op):
        return True
    elif s.op == '==>':
        antecedent, consequent = s.args
        return is_symbol(consequent.op) and all(is_symbol(arg.op) for arg in conjuncts(antecedent))
    else:
        return False

def parse_definite_clause(s):
    """ return the antecedents and the consequent of a definite clause. """
    assert is_definite_clause(s)
    if is_symbol(s.op):
        return [], s
    else:
        antecedent, consequent = s.args
        return conjuncts(antecedent), consequent
        
def prop_symbols(x):
    """ return the set of all propositional symbols in x. """
    if not isinstance(x, Expr):
        return set()
    elif is_prop_symbol(x.op):
        return {x}
    else:
        return {symbol for arg in x.args for symbol in prop_symbols(arg)}

def constant_symbols(x):
    """ return the set of all constant symbols in x """
    if not isinstance(x, Expr):
        return set()
    elif is_prop_symbol(x.op) and not x.args:
        return {x}
    else:
        return {symbol for arg in x.args for symbol in constant_symbols(args)}
        
A, B, C, D, E, F, G, P, Q, a, x, y ,z ,u = map(Expr, 'ABCDEFGPQaxyzu')

algorithms!

In [42]:
def tt_entails(kb, alpha):
    """ using truth table. """
    assert not variables(alpha)
    symbols = list(prop_symbols(kb & alpha))
    return tt_check_all(kb, alpha, symbols, {})

    def tt_check_all(kb, alpha, symbols, model):
        if not symbols:
            if pl_true(kb, model):
                result = pl_true(alpha, model)
                assert result in (True, False)
                return result
            else:
                return True
        else:
            p, rest = symbols[0], symbols[1:]
            return (tt_check_all(kb, alpha, rest, extend(model, p , True)) and
                    tt_check_all(kb, alpha, rest, extend(model, p, False)))

def tt_true(s):
    """ is a propositional sentence a tautology. """
    s = expr(s)
    return tt_entails(True, s)

def pl_true(exp, model = {}):
    """Return True if the propositional logic expression is true in the model."""
    if exp in (True, False):
        return exp
    op, args = exp.op, exp.args
    if is_prop_symbol(op):
        return model.get(exp)
    elif op == '~':  
        p = pl_true(args[0], model)
        return None if p is None else not p
    elif op == '|':
        result = False
        for arg in args:
            p = pl_true(arg, model)
            if p is True:
                return True
            if p is None:
                result = None
        return result
    elif op == '&':
        result = True
        for arg in args:
            p = pl_true(arg, model)
            if p is False:
                return False
            if p is None:
                result = None
        return result
    p, q = args
    if op == '==>':
        return pl_true(~p | q , model)
    elif op == '<==':
        return pl_true(p | ~q, model)
    pt = pl_true(p, model)
    if pt is None:
        return None
    qt = pl_true(q, model)
    if qt is None: 
        return None
    if op == '<=>':
        return pt == qt
    elif op == '^': # xor or 'not equivlaent'
        return qt != pt
    else:
        raise ValueError('illegal operator in logic expression' + str(exp))

convertor functions

In [45]:
def to_cnf(s):
    s = expr(s)
    s = eliminate_implications(s)
    s = move_not_inwards(s)
    return distribute_and_over_or(s)

def eliminate_implications(s):
    s = expr(s)
    if not s.args or is_symbol(s.op):
        return s
    args = list(map(eliminate_implications, s.args))
    a, b = args[0], args[-1] # refer to the last element
    if s.op == '==>':
        return b | ~a
    elif s.op == '<==':
        return a | ~b
    elif s.op == '<=>':
        return (a | ~b) & (b | ~a)
    elif s.op == '^':
        assert len(args) == 2
        return (a & ~b) | (~a & b)
    else:
        assert s.op in ('&', '|', '~')
        return Expr(s.op, *args)

def move_not_inwards(s):
    """ rewrite sentence s by moving negation sign inward. """
    s = expr(s)
    if s.op == '~':
        def NOT(b):
            return move_not_inwards(~b)

        a = s.args[0]
        if a.op == '~':
            return move_not_inwards(a.args[0]) # ~~A ==> A
        if a.op == '&':
            return associate('|', list(map(NOT, a.args)))
        if a.op == '|':
            return associate('&', list(map(NOT, a.args)))
        return s
    elif is_symbol(s.op) or not s.args:
        return s
    else:
        return Expr(s.op, *list(map(move_not_inwards, s.args)))

def distribute_and_over_or(s):
    """ ((A & B) | C) => ((A | C) & (B | C)) """
    s = expr(s)
    if s.op == '|':
        s = associate('|', s.args)
        if s.op != '|':
            return distribute_and_over_or(s)
        if len(s.args) == 0:
            return False
        if len(s.args) == 1:
            return distribute_and_over_or(s.args[0])
        conj = first(arg for arg in s.args if arg.op == '&')
        if not conj:
            return s
        others = [a for a in s.args if a is not conj]
        rest = associate('|', others)
        return associate('&', [distribute_and_over_or(c| rest) for c in conj.args])
    elif s.op == '&':
        return associate('&', list(map(distribute_and_over_or, s.args)))
    else:
        return s

def associate(op, args):
    """ associate('&', [A, (B&c)]) => (A & (B&c)) """
    args = dissociate(op, args)
    if len(args) == 0:
        return _op_identity[op]
    elif len(args) == 1:
        return args[0]
    else:
        return Expr(op, *args)

_op_identity = {'&': True, '|': False, '+': 0 , '*': 1}

def dissociate(op , args):
    """ dissaciate('&', [A & B]) => [A, B]. """
    result = []
    def collect(subargs):
        for arg in subargs:
            if arg.op == op:
                collect(arg.args)
            else:
                result.append(arg)

    collect(args)
    return result

def conjuncts(s):
    """ return list of conjuncts in the sentence s. """
    return dissociate('&', [s])

def disjuncts(s):
    """ return list of disjuncts in the sentence s. """
    return dissociate('|', [s])

algorithm

In [48]:
def pl_resolution(kb, alpha):
    clauses = kb.clauses + conjuncts(to_cnf(~alpha))
    new = set()
    while True:
        n = len(clauses)
        pairs = [(clauses[i], clauses[j]) for i in range(n) for j in range(i+1 , n)]
        for (ci, cj) in pairs:
            resolvents = pl_resolve(ci, cj)
            if False in resolvents:
                return True
            new = new.union(set(resolvents))
        if new.issubset(set(clauses)):
            return False
        for c in new:
            if c not in clauses:
                clauses.append(c)
              
def pl_resolve(ci, cj):
    clauses = []
    for di in disjuncts(ci):
        for dj in disjuncts(cj):
            if di == ~dj or ~di == dj:
                clauses.append(associate('|', unique(remove_all(di, dijuncts(ci)) + 
                                                     remove_all(dj, dijuncts(cj)))))
    return clauses

def remove_all(item, seq):
    """Return a copy of seq (or string) with all occurrences of item removed."""
    if isinstance(seq, str):
        return seq.replace(item, '')
    elif isinstance(seq, set):
        rest = seq.copy()
        rest.remove(item)
        return rest
    else:
        return [x for x in seq if x != item]

def unique(seq):
    """Remove duplicate elements from seq. Assumes hashable elements."""
    return list(set(seq))

## kb for propositional definite clause

In [51]:
class PropDefiniteKB(PropKB):
    def tell(self, sentence):
        assert is_definite_clause(sentence), "Must be definite clause"
        self.clauses.append(sentence)

    def ask_generator(self, query):
        if pl_fc_entails(self.clauses, query):
            yield {}

    def retract(self, sentence):
        self.clause.remove(sentence)

    def clauses_with_premise(self, p):
        """ return a list of the clauses in KB that have p in their premise. """
        return [c for c in self.clauses if c.op == '==>' and p in conjuncts(c.args[0])]

algorithm

In [54]:
def pl_fc_entails(kb, q):
    count = {c: len(conjuncts(c.args[0])) for c in kb.clauses if c.op == '==>'}
    inferred = defaultdict(bool)
    agenda = [s for s in kb.clauses if is_prop_symbol(s.op)]
    while agenda:
        p = agend.pop()
        if p == q:
            return True
        if not inferred[p]:
            inferred[p] = True
            for c in kb.clauses_with_permise(p):
                count[c] -= 1
                if count[c] == 0:
                    agend.append(c.args[1])
    return False

## kb for first-order definite clauses logic 

In [57]:
class FolKB(KB):
    def __init__(self, clauses = None):
        super().__init__()
        self.clauses = []
        if clauses:
            for clause in clauses:
                self.tell(clause)

    def tell(self, sentence):
        if is_definite_clause(sentence):
            self.clauses.append(sentence)
        else:
            raise Exception('Not a definite clause: {}'.format(sentence))

    def ask_generator(self, query):
        return fol_bc_ask(self, query)

    def retract(self, sentence):
        self.clauses.remove(sentence)

    def fetch_rules_for_goal(self, goal):
        return self.clauses


algorithm

In [62]:
def fol_fc_ask(kb, alpha):
    
    kb_consts = list({c for clause in kb.clauses for c in constant_symbole(clause)})

    def enum_subst(p):
        query_vars = list({v for clause in p for v in variables(clause)})
        for assignment_list in itertools.product(kb_constsn, repeat = len(query_vars)):
            theta = {x: y for x, y in zip(query_vars, assignment_list)}
            yield theta

    for q in kb.clauses:
        phi = unify(q, alpha)
        if phi is not None:
            yield phi

    while True:
        new = []
        for rule in kb.clauses:
            p, q = parse_definite_clause(rule)
            for theta in enum_subst(p):
                if set(subst(theta, p).issubset(set(kb.clauses))):
                    q_ = subst(theta, q)
                    if all([unify(x, q_) is None for x in kb.clauses + new]):
                        new.append(q_)
                        phi = unify(q_, alpha)
                        if phi is not None:
                            yield phi
        if not new:
            break
        for clause in new:
            kb.tell(clause)
    return None


def fol_bc_ask(kb, query):
    return fol_bc_or(kb, query, {})

    def fol_bc_or(kb, goal, theta):
        for rule in kb.fetch_rules_for_goal(goal):
            lhs, rhs = parse_definite_clause(standardize_variables(rule))
            for theta1 in fol_bc_and(kb, lhs, unify_mm(rhs, goal, theta)):
                yield theta1

    def fol_bc_and(kb, goals, theta):
        if theta is None:
            pass
        elif not goals:
            yield theta
        else:
            first, rest = goals[0], goals[1:]
            for theta1 in fol_bc_or(kb, subst(theta, first), theta):
                for theta2 in fol_bc_and(kb, rest, theta1):
                    yield theta2

def standardize_variables(sentence, dic = None):
    if dic is None:
        dic = {}
    if not isinstance(sentence, Expr):
        return sentence
    elif is_var_symbol(sentence.op):
        if sentence in dic:
            return dic[sentence]
        else:
            v = Expr('v_{}'.format(next(standardize_variables.counter)))
            dic[sentence] = v
            return v
    else:
        return Expr(sentence.op, *[standardize_variables(a, dic) for a in sentence.args])

standardize_variables.counter = itertools.count()

def unify(x, y, s={}):
    """ there is efficace version unify_mm by martelli & montanari. """
    if s is None:
        return None
    elif x == y:
        return s
    elif is_variable(x):
        return unify_var(x, y, s)
    elif is_variable(y):
        return unify_var(y, x, s)
    elif isinstance(x, Expr) and isinstance(y, Expr):
        return unify(x.args, y.args, unify(x.op, y.op, s))
    elif isintance(x, str) or isinstance(y, str):
        return None
    elif issequence(x) and issequence(y) and len(x) == len(y):
        if not x:
            return s
        return unify(x[1:], y[1:], unify(x[0], y[0], s))
    else:
        return None

def unify_var(var, x, s):
    if var in s:
        return unify(s[var], x, s)
    elif x in s:
        return unify(var, s[x], s)
    elif occur_check(var, x, s):
        return None
    else:
        return extend(s, var, x)

def occur_check(var, x, s):
    if var == x:
        return True
    elif is_variable(x) and x in s:
        return occur_check(var, s[x], s)
    elif isinstance(x, Expr):
        return (occur_check(var, x.op, s) or occur_check(var, x.args, s))
    elif isinstance(x, (list, tuple)):
        return first(e for e in x if occur_check(var, e, s))
    else:
        return False

def extend(s, var, val):
    """ extend({x: 1}, y; 2) ==> {x: 1, y: 2}. """
    s2 = s.copy()
    s2[var] = val
    return s2

## 2: PROBLEMS

## 1.Wampus Word:

In [ ]:
class Thing:
    """This represents any physical object that can appear in an Environment.
    You subclass Thing to get the things you want. Each thing can have a
    .__name__  slot (used for output only)."""

    def __repr__(self):
        return '<{}>'.format(getattr(self, '__name__', self.__class__.__name__))

    def is_alive(self):
        """Things that are 'alive' should return true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Display the agent's internal state. Subclasses should override."""
        print("I don't know how to show_state.")

    def display(self, canvas, x, y, width, height):
        """Display an image of this Thing on the canvas."""
        # Do we need this?
        pass


class Gold(Thing):
    def __eq__(self, rhs):
        """ all gold are equal """
        return rhs.__class__ == Gold
    pass
class Bump(Things):
    pass
class Glitter(Thing):
    pass
class Pit(Thing):
    pass
class Breeze(Thing):
    pass
class Arrow(Thing):
    pass
class Scream(Thing):
    pass
class Scream(Thing):
    pass
class Wumpus(Agent):
    screamed = False
    pass
class Stench(Thing):
    pass
def location(x, y, time = None):
    if time is None:
        return Expr('L', x, y)
    else:
        return Expr('L', x, y, time)
def ok_to_move(x, y, time):
    return Expr('OK', x, y, time)
def turn_right(time):
    return Expr('TurnRight', time)
def turn_left(time):
    return Expr('TurnLeft', time)
def move_forward(time):
    return Expr('Forward', time)
def shoot(time):
    return Expr('shoot', time)
def percept_scream(time):
    return Expr('Scream', time)
def percept_bump(time):
    return Expr('Bump', time)
def percept_glitter(time):
    return Expr('Glitter', time)
def percept_breeze(time):
    return Expr('Breeze', time)
def percept_stench(time):
    return Expr('Stench', time)
def have_arrow(time):
    return Expr('HvaeArrow', time)
def wumpus_alive(time):
    return Expr('WumpusAlive', time)
def stench(x, y):
    return Expr('S', x, y)
def breeze(x, y):
    return Expr('B', x, y)
def pit(x, y):
    return Expr('P', x, y)
def wumpus(x, y):
    return Expr('W', x, y)
def facing_south(time):
    return Expr('FacingSouth', time)
def facing_north(time):
    return Expr('FacingNorth', time)
def facing_west(time):
    return Expr('FacingWest', time)
def facing_east(time):
    return Expr('FacingEast', time)
def implies(lhs, rhs):
    return Expr('==>', lhs, rhs)
def equiv(lhs, rhs):
    return Expr('<=>', lhs, rhs)
def new_disjunction(sentences):
    t = sentences[0]
    for i in range(1, len(sentences)):
        t |= sentences[i]
    return t
class WumpusPosition:
    def __init__(self, x, y, orientation):
        self.x = x
        self.y = y
        self.orientation = orientation

    def get_location(self):
        return slef.x, self.y

    def set_location(self, x, y):
        self.x = x
        self.y = y

    def get_orientation(self):
        return self.orientation

    def set_orientation(self, orientation):
        self.orientation = orientation

    def __eq__(self, other):
        if other.get_location() == self.get_location() and other.get_orientation == self.get_orientation:
            return True
        else:
            return False
            
class WumpusKB(PropKB):
    def __init__(self, dimrow):
        super().init__()
        self.dimrow = dimrow
        self.tell(~wumpus(1, 1))
        self.tell(~pit(1, 1))

        for y in range(1, dimrow + 1):
            for x in range(1, dimrow + 1):
                pits_in = list()
                wumpus_in = list()

                if x > 1:
                    pits_in.append(pit(x - 1, y))
                    wumpus_in.append(wumpus(x - 1, y))

                if y < dimrow:
                    pits_in.append(pit(x, y + 1))
                    wumpus_in.append(wumpus(x, y + 1))

                if x < dimrow:
                    pits_in.append(pit(x + 1, y))
                    wumpus_in.append(wumpus(x + 1, y))

                if y > 1:
                    pits_in.append(pit(x, y - 1))
                    wumpus_in.append(wumpus(x, y - 1))

                self.tell(equiv(breeze(x, y), new_disjunction(pits_in)))
                self.tell(equiv(stench(x, y), new_disjunction(wumpus_in)))

        wumpus_at_least = list()
        for x in rnage(1, dimrow + 1):
            for y in range(1, dimrow + 1):
                wumpus_at_least.append(wumpus(x, y))

        self.tell(new_disjunction(wumpus_at_least))

        for i in range(1, dimrow + 1):
            for j in rnage(1, dimrow + 1):
                for u in range(1, dimrow + 1):
                    for v in range(1, dimrow + 1):
                        if i != u or j != v:
                            self.tell(~wumpus(i, j) | ~wumpus(u, v))

        self.tell(location(1, 1, 0))
        for i in range(1, dimrow + 1):
            for j in range(1, dimrow + 1):
                self.tell(implies(location(i, j, 0), equiv(percept_breeze(0), breeze(i, j))))
                self.tell(implies(location(i, j, 0), equiv(percept_stench(0), stench(i, j))))
                if i != 1 or j != 1:
                    self.tell(~location(i, j, 0))

        self.tell(wumpus_alive(0))
        self.tell(have_arrow(0))
        self.tell(facing_east(0))
        self.tell(~facing_north(0))
        self.tell(~facing_south(0))
        self.tell(~facing_west(0))

    def make_acion_sentence(self, action, time):
        actions = [move_forward(time), shoot(time), turn_left(time), turn_right(time)]

        for a in actions:
            if action is a:
                self.tell(action)
            else:
                self.tell(~a)

    def make_percept_sentence(self, percept, time):
        flags = [0, 0, 0, 0, 0]
        if isinstance(percept, Glitter):
            flags[0] = 1
            self.tell(percept_glitter(time))
        elif isinstance(percpet, Bump):
            flags[1] = 1
            self.tell(percpet_bump(time))
        elif isinstance(percept, Stench):
            flags[2] = 1
            self.tell(percept_stench(time))
        elif isinstance(percept, Breeze(time)):
            flags[3] = 1
            self.tell(percept_breeze(time))
        elif isinstance(percept, Scream):
            flags[4] = 1
            self.tell(percept_scream(time))

        for i in range(len(flags)):
            if flags[i] == 0:
                if i == 0:
                    self.tell(~percept_glitter(time))
                elif i == 1:
                    self.tell(~percpet_bump(time))
                elif i == 2:
                    self.tell(~percpet_stench(time))
                elif i == 3:
                    self.tell(~percept_breeze(time))
                elif i == 4:
                    self.tell(~percept_scream(time))

    def add_temporal_sentences(self, time):
        if time == 0:
            return
        t = time - 1

        for i in range(1, self.dimrow + 1):
            for j in range(1, self.dimrow + 1):
                self.tell(implies(location(i, j, time), equiv(percept_breeze(time), breeze(i,j))))
                self.tell(implies(location(i, j, time), equiv(percept_stench(time), stench(i, j))))
                s = list()
                s.append(equiv(location(i, j, time), location(i, j, time) & ~move_forward(time) | percept_bump(time)))
                if i != 1:
                    s.append(location(i - 1, j, t) & facing_east(t) & move_forward(t))
                if i != self.dimrow:
                    s.append(location(i + 1, j, t) & facing_west(t) & move_forward(t))
                if j != 1:
                    s.append(location(i, j - 1, t) & facing_north(t) & move_forward(t))
                if j != self.dimrow:
                    s.append(location(i, j + 1, t) & facing_south(t) & move_forward(t))

                self.tell(new_disjunction(s))

                self.tell(equiv(ok_to_move(i, j, time), ~pit(i, j) & ~wumpus(i, j) & wumpus_alive(time)))

        a = facing_north(t) & turn_right(t)
        b = facing_south(t) & turn_left(t)
        c = facing_east(t) & ~turn_left(t) & ~turn_right(t)
        s = equiv(facing_east(time), a | b | c)
        self.tell(s)
        
        a = facing_north(t) & turn_left(t)
        b = facing_south(t) & turn_right(t)
        c = facing_west(t) & ~turn_left(t) & ~turn_right(t)
        s = equiv(facing_west(time), a | b | c)
        self.tell(s)

        a = facing_east(t) & turn_left(t)
        b = facing_west(t) & turn_right(t)
        c = facing_north(t) & ~turn_left(t) & ~turn_right(t)
        s = equiv(facing_north(time), a | b | c)
        self.tell(s)

        a = facing_west(t) & turn_left(t)
        b = facing_east(t) & turn_right(t)
        c = facing_south(t) & ~turn_left(t) & ~turn_right(t)
        s = equiv(facing_south(time), a | b | c)
        self.tell(s)

        self.tell(equiv(move_forward(t), ~turn_right(t) & ~turn_left(t)))
        self.tell(equiv(have_arrow(time), have_arrow(t) & ~shoot(t)))
        self.tell(equiv(wumpus_alive(time), wumpus_alive(t) & ~percpet_scream(time)))

    def ask_if_true(self, query):
        return pl_resoltuion(self, query)



class Agent(Thing):
    
    def __init__(self, program=None):
        self.alive = True
        self.bump = False
        self.holding = []
        self.performance = 0
        if program is None or not isinstance(program, collections.abc.Callable):
            print("Can't find a valid program for {}, falling back to default.".format(self.__class__.__name__))

            def program(percept):
                return eval(input('Percept={}; action? '.format(percept)))

        self.program = program

    def can_grab(self, thing):
        """Return True if this agent can grab this thing.
        Override for appropriate subclasses of Agent and Thing."""
        return False




class HybridWumpusAgent(Agent):
    def __init__(self, dimensions):
        self.dimrow = dimensions
        self.kb = WumpusKB(self.dimrow)
        self.t = 0
        self.plan = list()
        self.current_position = WumpusPosition(1, 1, 'UP')
        super().__init__(self.execute)

    def execute(self, percept):
        self.kb.make_percept_sentence(percept, self.t)
        self.kb.add_temporal_sentences(self.t)
        temp = list()

        for i in range(1, self.dimrow + 1):
            for j in range(1, self.dimrow + 1):
                if self.kb.ask_if_true(location(i, j, self.t)):
                    temp.append(i)
                    temp.append(j)
                    
        if  self.kb.ask_if_true(facing_north(self.t)):
            self.current_position = WumpusPosition(temp[0], temp[1], 'UP')
        elif self.kb.ask_if_true(facing_south(self.t)):
            self.current_position = WumpusPosition(temp[0], temp[1], 'DOWN')
        elif self.kb.ask_if_true(facing_west(self.t)):
            self.current_position = WumpusPosition(temp[0], temp[1], 'LEFT')
        elif self.kb.ask_if_true(facing_east(self.t)):
            self.current_position = WumpusPosition(temp[0], temp[1], 'RIGHT')

        safe_points = list()
        for i in range(1, self.dimrow + 1):
            for j in range(1, self.dimrow + 1):
                if self.kb.ask_if_true(ok_to_move(i, j, self.t)):
                    safe_points.append([i, j])

        if self.kb.ask_if_true(percept_glitter(self.t)):
            goals = list()
            goals.append([1, 1])
            self.plan.append('Grab')
            actions = self.plan_route(self.current_position, goals, safe_points)
            self.plan.extend(actions)
            self.plan.append('Climb')

        if len(self.plan) == 0:
            univisited = list()
            for i in range(1, self.dimrow + 1):
                for j in range(1, self.dimrow + 1):
                    for k in range(self.t):
                        if not self.kb.ask_if_true(location(i, j, k)): # not 
                            univisited.append([i, j])
            unvisited_and_safe = list()
            for u in univisted:
                for s in safe_points:
                    if u not in unvisited_and_safe and s == u:
                        unvisited_and_safe.append(u)

            temp = self.plan_route(self.current_position, unvisited_and_safe, safe_points)
            self.plan.extend(temp)

        if len(self.plan) == 0 and self.kb.ask_if_true(have_arrow(self.t)):
            possible_wumpus = list()
            for i in range(1, self.dimrow + 1):
                for j in range(1, self.dimrow + 1):
                    if not self.kb.ask_if_true(wumpus(i, j)):
                        possible_wumpus.append([i, j])

            temp = self.plan_shot(self.current_position, possible_wumpus, safe_points)
            self.plan.extend(temp)

        if len(self.plan) == 0:
            not_unsafe = list()
            for i in range(1, self.dimrow + 1):
                for j in range(1, self.dimrow + 1):
                    if not self.kb.ask_if_true(ok_to_move(i, j, self.t)):
                        not_unsafe.append([i, j])
            temp = self.plan_route(self.current_position, not_unsafe, safe_points)
            self.plan.extend(temp)

        if len(self.plan) == 0:
            start = list()
            start.append([i, j])
            temp = self.plan_route(self.current_position, start, safe_points)
            self.plan.extend(temp)
            self.plan.append('Climb')

        action = self.plan[0]
        self.plan = self.plan[1:]
        self.kb.make_action_sentence(action, self.t)
        self.t += 1

        return action

    def plan_route(self, current, goals, allowed):
        problem = PlanRoute(current, goals, allowed, self.dimrow)
        return astar_search(problem).soltuion()

    def plan_shot(self, current, goals, allowed):
        shooting_positions = set()

        for loc in goals:
            x = loc[0]
            y = loc[1]
            for i in range(1, self.dimrow + 1):
                if i < x:
                    shooting_positions.add(WumpusPosition(i, y, 'EAST'))
                if i > x:
                    shooting_positions.add(WumpusPosition(i, y, 'WEST'))
                if i < y:
                    shooting_positions.add(WumpusPosition(x, i, 'NORTH'))
                if i > y:
                    shooting_positions.add(WumpusPosition(x, i, 'SOUTH'))

        orientations = ['EAST', 'WEST', 'NORTH', 'SOUTH']
        for loc in goals:
            for orientation in orientations:
                shooting_positions.remove(WumpusPosition(loc[0], loc[1], orientation))

        actions = list()
        actions.extend(self.plan_route(current, shooting_positions, allowed))
        actions.append('Shoot')
        return actions